In [ ]:
import time
import numpy as np
import matplotlib.pyplot as plt

num_bins=50

# I. Load and plot

In [ ]:
Scenario    = 'Case3'
path_prefix = './data/' + Scenario + '_'

# Loading data
with open(path_prefix + 'first_kind.npy', 'rb') as f:
    zeroes_first_kind = np.load(f)
with open(path_prefix + 'second_kind.npy', 'rb') as f:
    zeroes_second_kind = np.load(f)
with open(path_prefix + 'critical_points.npy', 'rb') as f:
    critical_points = np.load(f)
with open(path_prefix + 'branch_points.npy', 'rb') as f:
    branch_points = np.load(f)

N = len( zeroes_first_kind )
p = N

# Histogram of singular values
fig, ax = plt.subplots()
n, bins, patches = ax.hist(zeroes_first_kind, num_bins, density=True)
ax.set_xlabel('Eigenvalues')
ax.set_ylabel('Probability density')
ax.set_title(r'Histogram of singular values for N={}'.format(N))
fig.tight_layout()
plt.show()

In [ ]:
# Plotting critical points
def normalize( array ):
    min = np.min( array )
    max = np.max( array )
    return (array-min)/(max-min)
def transport_to_uniform( array ):
    uniform_sample = np.linspace( 0, 1, len(array))
    sorted_indices = np.argsort( array )
    return uniform_sample[ sorted_indices ]
critical_points_color = transport_to_uniform( np.abs( branch_points ) )
plt.figure( figsize=(20,10) )
plt.scatter( np.real(critical_points), np.imag(critical_points), marker='*', c=critical_points_color, label=f'Critical points (p-1={p-1})')
plt.scatter( np.real(zeroes_first_kind), np.imag(zeroes_first_kind), marker='*', c='r', label=f'Eigenvalues (p={p})')
plt.scatter( np.real(zeroes_second_kind), np.imag(zeroes_second_kind), marker='*', c='b', label=f'Zeroes of the second kind (p-1={p-1})')
plt.title( 'Eigenvalues and critical points')
plt.legend()
plt.colorbar()
plt.show()

# Plotting branch points
## Transport colors to uniform
branch_points_color = transport_to_uniform( np.abs( np.imag( critical_points ) ) )
## Plots as usual
plt.figure( figsize=(10,20) )
plt.scatter( np.real(branch_points),  np.imag(branch_points), marker='*', c=branch_points_color, label=f'Branch points (p-1={p-1})')
plt.scatter( np.real(branch_points), -np.imag(branch_points), marker='x', c=branch_points_color, label=f'(Conjugate) Branch points (p-1={p-1})')
plt.title( 'Branch points $m = M_\mu(z)$')
plt.legend()
plt.colorbar()
plt.show()

In [ ]:
import freeDeconvolution

mu_observed = freeDeconvolution.core.DiscreteMeasure( zeroes_first_kind, None)
mu_observed.zeroes_second_kind = zeroes_second_kind

c, _, population_spectrum, interval_cdf, population_cdf = freeDeconvolution.scenarios.initiate_scenario( Scenario, N)
r = (1+np.sqrt(c))**2 #Right end of MP
l = (1-np.sqrt(c))**2 #Left end of MP
print("c :", c)

mu_signal   = freeDeconvolution.core.DiscreteMeasure( population_spectrum, None)

# Warning: This has changed to fit El Karoui's Case 1
def G_theoretical(z):
    sqrt = np.sqrt( (z-c-1)*(z-c-1)-4*c )
    positive_imag = (np.imag(sqrt)>0)
    sqrt = positive_imag*sqrt - (1-positive_imag)*sqrt
    return (z+c-1-sqrt)/(2*z*c)

if Scenario=='Case1':
    interval = np.linspace(-1, 3, 100)
    density1 = -np.imag( G_theoretical(interval+0.05j) )/np.pi
    density2 = -np.imag( mu_observed.G_empirical(interval+0.05j) )/np.pi
    plt.plot( interval, density1, label='Theoretical')
    plt.plot( interval, density2, label='Empirical')
    plt.legend()
    plt.show()

# II. Geometry of Riemann surface

In [ ]:
# Find highest critical point
highest_point_index = np.argmax( np.imag(critical_points) )
highest_point = critical_points[highest_point_index]

# Form unit circle
contour_type = "circle"
mesh_size    = int(2e5)
interval = np.linspace(0, 2*np.pi, mesh_size)
unit_circle = np.cos(interval) + np.sin(interval)*1.0j


critical_points_color = transport_to_uniform( np.abs( branch_points ) )
plt.figure( figsize=(20,10) )
z_circle = highest_point + 0.04*unit_circle
x = np.real(z_circle)
y = np.imag(z_circle)
plt.plot( x, y, c='r')
plt.scatter( np.real(critical_points), np.imag(critical_points), marker='*', c=critical_points_color, label=f'Critical points (p-1={p-1})')
plt.scatter( np.real(zeroes_first_kind), np.imag(zeroes_first_kind), marker='*', c='r', label=f'Eigenvalues (p={p})')
plt.scatter( np.real(zeroes_second_kind), np.imag(zeroes_second_kind), marker='*', c='b', label=f'Zeroes of the second kind (p-1={p-1})')
plt.title( 'Eigenvalues and critical points')
plt.legend()
plt.colorbar()
plt.show()

# Plotting branch points
## Transport colors to uniform
branch_points_color = transport_to_uniform( np.abs( np.imag( critical_points ) ) )
## Plots as usual
plt.figure( figsize=(10,20) )
m_circle = mu_observed.M_empirical( z_circle )
x = np.real(m_circle)
y = np.imag(m_circle)
plt.plot( x, y, c='r', label='Image path in m space')
m2_circle = m_circle/(0.3*m_circle + 1)
x = np.real(m2_circle)
y = np.imag(m2_circle)
plt.plot( x, y, c='orange')
plt.scatter( np.real(branch_points),  np.imag(branch_points), marker='*', c=branch_points_color, label=f'Branch points (p-1={p-1})')
plt.scatter( np.real(branch_points), -np.imag(branch_points), marker='x', c=branch_points_color, label=f'(Conjugate) Branch points (p-1={p-1})')
plt.title( 'Branch points $m = M_\mu(z)$')
plt.ylim( (-2, 2) )
plt.legend()
plt.colorbar()
plt.show()

## III. Inversion without homotopy

In [ ]:
def newton_raphson( f, f_prime, initial, max_iter=12, tol=1e-12):
    z = initial
    k = 0
    while k<max_iter:
        value = f(z)
        derivative_value = f_prime(z)
        z = z - value/derivative_value
        if abs(value)<tol:
            break
        k = k + 1
    return z

In [ ]:
# Find the z_target such that M(z_target) = m_target
empirical_mean = zeroes_first_kind.mean()
m_array = np.linspace( 0.1, 20.0, 1000 )*(-1j) # Breaks at 0.7
z_array = []
debug = False
for m in m_array:
    m_target  = np.array( [m] )
    if len(z_array)>0:
        z_initial = z_array[-1]
    else:
        z_initial = empirical_mean/m_target
    z_result  = newton_raphson( lambda z : mu_observed.M_empirical(z)-m_target, mu_observed.M_prime_empirical, z_initial)
    z_array.append( z_result )
    if debug:
        print( "Result: ", z_result)
        print( "Error: ", np.abs( m_target - mu_observed.M_empirical(z_result)) )
z_array = np.array( z_array )

m_final = m_array[-1]
z_final = z_array[-1]

# Plots in z and m space
fig, axs = plt.subplots(1, 2, figsize=(20, 10))

axs[0].plot( np.real(z_array), np.imag(z_array), label=f'Lift in z space)')
axs[0].set_title( 'z space')

axs[1].plot( np.real(m_array), np.imag(m_array), label=f'Path in m space)')
axs[1].set_title( 'm space')

plt.legend()
plt.show()

In [ ]:
# Find the z_target such that M(z_target) = m_target
empirical_mean = zeroes_first_kind.mean()
radius = 1.4
phase  = -np.pi/2 
interval = np.linspace( phase, phase + 2*np.pi, 100 )
ellipse  = np.cos( interval ) + 1.0*np.sin( interval )*1j
m_array = radius*ellipse
z_array = []
debug = False
for m in m_array:
    m_target  = np.array( [m] )
    z_initial = empirical_mean/m_target
    if len(z_array)>0:
        z_initial = z_array[-1]
    else:
        z_initial = z_final
    z_result  = newton_raphson( lambda z : mu_observed.M_empirical(z)-m_target, mu_observed.M_prime_empirical, z_initial)
    z_array.append( z_result )
    if debug:
        print( "Result: ", z_result)
        print( "Error: ", np.abs( m_target - mu_observed.M_empirical(z_result)) )
z_array = np.array( z_array )

# Plots in z and m space
fig, axs = plt.subplots(1, 2, figsize=(20, 10), sharey=False)

axs[0].plot( np.real(z_array), np.imag(z_array), label=f'Lift in z space)')
axs[0].scatter( np.real(critical_points), np.imag(critical_points), marker='*', c='g', label=f'Critical points (N-1={N-1})')
axs[0].scatter( np.real(critical_points), -np.imag(critical_points), marker='*', c='g', label=f'Critical points (N-1={N-1})')
axs[0].scatter( np.real(zeroes_first_kind), np.imag(zeroes_first_kind), marker='*', c='r', label=f'Eigenvalues (N={N})')
axs[0].scatter( np.real(zeroes_second_kind), np.imag(zeroes_second_kind), marker='*', c='b', label=f'Zeroes of the second kind (N-1={N-1})')
axs[0].set_title( 'z space')

axs[1].plot( np.real(m_array), np.imag(m_array), label=f'Path in m space)')
axs[1].scatter( np.real(branch_points),  np.imag(branch_points), marker='*', c=branch_points_color, label=f'Branch points (N-1={N-1})')
axs[1].scatter( np.real(branch_points), -np.imag(branch_points), marker='x', c=branch_points_color, label=f'(Conjugate) Branch points (N-1={N-1})')
axs[1].set_title( 'm space')
axs[1].set_ylim( -5, 5)

plt.legend()
plt.show()

In [ ]:
from freeDeconvolution import boxes

def box_to_path( box, mesh_size):
       interval =  np.linspace( 0,1, mesh_size)
       path = []
       for segment in boxes.box_segments_enum:
              vector = box[ segment[1] ] - box[ segment[0] ]
              origin = box[ segment[0] ]
              s = origin + interval*vector
              #
              path = path + list(s)
       return path

radius = np.max( np.imag(critical_points) )*1.2
box = {
       'top_left'    : np.min( zeroes_first_kind ) - 0.1 + radius*1.0j,
       'bottom_right': np.max( zeroes_first_kind ) + 0.1 - radius*1.0j,
}
bounding_box = boxes.extend_box( box )
path = box_to_path( box, int(1e5) )
plt.plot( np.real(path), np.imag(path), c='r')
plt.show()

z_array = np.array( path )
m_array = mu_observed.M_empirical( z_array )

# Plots in z and m space
fig, axs = plt.subplots(1, 2, figsize=(20, 10), sharey=False)

z_circle = highest_point + 0.02*unit_circle
x = np.real(z_circle)
y = np.imag(z_circle)
axs[0].plot( x, y, c='r')

axs[0].plot( np.real(z_array), np.imag(z_array), label=f'Lift in z space)')
axs[0].scatter( np.real(critical_points), np.imag(critical_points), marker='*', c='g', label=f'Critical points (N-1={N-1})')
axs[0].scatter( np.real(critical_points), -np.imag(critical_points), marker='*', c='g', label=f'Critical points (N-1={N-1})')
axs[0].scatter( np.real(zeroes_first_kind), np.imag(zeroes_first_kind), marker='*', c='r', label=f'Eigenvalues (N={N})')
axs[0].scatter( np.real(zeroes_second_kind), np.imag(zeroes_second_kind), marker='*', c='b', label=f'Zeroes of the second kind (N-1={N-1})')
axs[0].set_title( 'z space')

#

m_circle = mu_observed.M_empirical( z_circle )
x = np.real(m_circle)
y = np.imag(m_circle)
axs[1].plot( x, y, c='r', label='Image path in m space')

axs[1].plot( np.real(m_array), np.imag(m_array), label=f'Path in m space)')
axs[1].scatter( np.real(branch_points),  np.imag(branch_points), marker='*', c=branch_points_color, label=f'Branch points (N-1={N-1})')
axs[1].scatter( np.real(branch_points), -np.imag(branch_points), marker='x', c=branch_points_color, label=f'(Conjugate) Branch points (N-1={N-1})')
axs[1].set_title( 'm space')
axs[1].set_ylim( -5, 5)

plt.legend()
plt.show()

In [ ]:
s_array = (1+m_array)/(m_array*z_array)

if Scenario=='Case1':
    c_interval = np.linspace(0.1, 0.5, 100)
    error_array = []
    for c_ in c_interval:
        error = np.abs( s_array*(c_*m_array+1)-1).max()
        error_array.append( error )
    error_array = np.array( error_array )
    plt.plot( c_interval, error_array )
    plt.show()

### Formulas

$$ m = zg - 1$$
$$ s = \frac{1+m}{m z}
   \Leftrightarrow
   sz - 1 = \frac{1}{m}
$$

In [ ]:
print( c )
# if Scenario=='Case1':
#     c=0.3
# elif Scenario=='Case2':
#     c=0.3
# elif Scenario=='Case3':
#     c=0.2

s_signal_array = s_array
s_noise_array  = 1/(c*m_array + 1)
s_deconv_array = s_signal_array/s_noise_array
m_deconv_array = 1/( s_deconv_array*z_array - 1)
m_deconv_theoretical_array = mu_signal.M_empirical( z_array )

# New take
z_deconv_array = z_array/(c*m_array+1)
m_deconv_array2 = mu_signal.M_empirical( z_deconv_array )

# Plots in z and m space
fig, axs = plt.subplots(1, 2, figsize=(20, 10), sharey=False)

axs[0].plot( np.real(z_array)       , np.imag(z_array)       , c='b', label=f'Original path in z space)')
axs[0].plot( np.real(z_deconv_array), np.imag(z_deconv_array), c='r', label=f'Deconv path in z space)')
axs[0].scatter( np.real(critical_points), np.imag(critical_points), marker='*', c='g', label=f'Critical points (N-1={N-1})')
axs[0].scatter( np.real(critical_points), -np.imag(critical_points), marker='*', c='g', label=f'Critical points (N-1={N-1})')
axs[0].scatter( np.real(zeroes_first_kind), np.imag(zeroes_first_kind), marker='*', c='r', label=f'Eigenvalues (N={N})')
axs[0].scatter( np.real(zeroes_second_kind), np.imag(zeroes_second_kind), marker='*', c='b', label=f'Zeroes of the second kind (N-1={N-1})')
axs[0].set_title( 'z space')

axs[1].plot( np.real(m_deconv_theoretical_array), np.imag(m_deconv_theoretical_array), c='b', label=f'Ground truth in m space')
axs[1].plot( np.real(m_deconv_array2), np.imag(m_deconv_array2), c='r', label=f'Ground truth: M of signal')
axs[1].plot( np.real(m_array), np.imag(m_array), label=f'Empirical path in m space')
axs[1].plot( np.real(m_deconv_array), np.imag(m_deconv_array), label=f'Empirical path in m space')
axs[1].scatter( np.real(branch_points),  np.imag(branch_points), marker='*', c=branch_points_color, label=f'Branch points (N-1={N-1})')
axs[1].scatter( np.real(branch_points), -np.imag(branch_points), marker='x', c=branch_points_color, label=f'(Conjugate) Branch points (N-1={N-1})')
axs[1].set_title( 'm space')
#axs[1].set_ylim( -7, 7)

plt.legend()
plt.show()

# IV. Deconvolution 

In [ ]:
# New take
z_array = z_deconv_array
m_deconv_array = m_deconv_array2
#
g_deconv_array = (m_deconv_array+1)/z_array
g_deconv_theoretical = (m_deconv_theoretical_array+1)/z_array

#resolution = 1/bounding_box['height']
resolution = 0.1*2*np.pi/bounding_box['height']
dz_array = z_array-np.roll(z_array, shift=1)
empirical_cdf = []
for x in interval_cdf:
    sigmoid = 1/(1+np.exp(resolution*(z_array-x)))
    value = g_deconv_array*sigmoid*dz_array
    value = value.sum()/(2*np.pi*1.0j)
    empirical_cdf.append( value )
# end for
empirical_cdf = np.real( np.array( empirical_cdf ) )

smoothed_population_cdf = []
for x in interval_cdf:
    sigmoid = 1/( 1 + np.exp(resolution*(population_spectrum-x)) )
    value = sigmoid.mean()
    smoothed_population_cdf.append( value )
# end for
smoothed_population_cdf = np.array( smoothed_population_cdf )


plt.figure( figsize=(10,5) )
plt.plot( interval_cdf, population_cdf, label='Asymptotic ground truth = Theoretical cdf of deconvolved signal' )
plt.plot( interval_cdf, empirical_cdf, label=f'''Empirical cdf of deconvolved signal and blur at resolution {1/resolution}''' )
plt.plot( interval_cdf, smoothed_population_cdf, label='Theoretical cdf of deconvolved signal at same resolution' )
plt.title( "CDF")
plt.legend()
plt.show()

def array_increment(array):
    return array[1:]-array[:-1]

plt.figure( figsize=(10,5) )
plt.plot( interval_cdf[:-1], array_increment(population_cdf), label='Asymptotic ground truth = Theoretical density of deconvolved signal' )
plt.plot( interval_cdf[:-1], array_increment(empirical_cdf), label=f'''Empirical density of deconvolved signal and blur at resolution {1/resolution}''' )
plt.title( "Densities")
plt.legend()
plt.show()

# V. OPRL reconstruction

In [ ]:
# Compute moments
# Input: z_array, g_deconv_array

moments_count = 19
dz_array  = z_array-np.roll(z_array, shift=1)
mom_array = np.zeros( moments_count + 1)
for mom_index in range( len(mom_array) ):
    value = g_deconv_array*(z_array**mom_index)*dz_array
    value = value.sum()/(2*np.pi*1.0j)
    print(value)
    mom_array[ mom_index ] = np.real(value)
# end for
print( "List of moments: ")
print( mom_array )

In [ ]:
import freeDeconvolution

# Compute
jacobi_a, jacobi_b = freeDeconvolution.oprl.jacobi_from_moments( mom_array )

# Print Jacobi coefficients
print( "Computed Jacobi coefficients:")
print( "b: ", jacobi_b )
print( "a: ", jacobi_a )
print( "")

In [ ]:
# Quadrature measure from our code
print("Quadrature measure")
support, weights = freeDeconvolution.quadrature_from_jacobi( jacobi_a, jacobi_b)
print( support )
print( weights )

In [ ]:
# Plot
space_array   = np.linspace(-1, 7, 100)
empirical_cdf = np.zeros_like( space_array )
for index in range(len(support)):
    eigenvalue = support[ index ]
    empirical_cdf += weights[index] * ( interval_cdf >= eigenvalue )
# end for
empirical_cdf = np.array( empirical_cdf )

plt.figure( figsize=(10,5) )
plt.plot( interval_cdf, population_cdf, label='Asymptotic ground truth = Theoretical cdf of deconvolved signal' )
plt.plot( interval_cdf, np.real(empirical_cdf), label=f'''Empirical cdf of deconvolved signal with OPRL reconstruction''' )
plt.legend()
plt.show()